# Week 1 - Exercice 3

Rework outparsers to smaller tasks then review LCEL and RetryWithErrorOutputParser

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers.retry import RetryWithErrorOutputParser, NAIVE_RETRY_WITH_ERROR_PROMPT

import re

In [2]:
import sys

sys.path.append("../")

import agents.technical_analyst.technical_analysis_pydantic_model as _pydantic_models

In [3]:
MODEL = "cogito:8b"
# MODEL = "adeelahmad/ReasonableLLAMA-Jr-3b:latest"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"
invocation = {"stock": "APPLE", "timeframe": "5 minutes"}

In [4]:
RETRY_PROMPT_TEMPLATE = NAIVE_RETRY_WITH_ERROR_PROMPT.template.split("\n")
RETRY_PROMPT_TEMPLATE = RETRY_PROMPT_TEMPLATE[:-1] + ["YOU MUST RESPECT THE SCHEMA PROVIDED IN THE PROMPT."] + RETRY_PROMPT_TEMPLATE[-1:]
RETRY_PROMPT_TEMPLATE = DEEP_THINKING_INSTRUCTION + "\n".join(RETRY_PROMPT_TEMPLATE)
RETRY_PROMPT_TEMPLATE

'Enable deep thinking subroutine.\n\nPrompt:\n{prompt}\nCompletion:\n{completion}\n\nAbove, the Completion did not satisfy the constraints given in the Prompt.\nDetails: {error}\nYOU MUST RESPECT THE SCHEMA PROVIDED IN THE PROMPT.\nPlease try again:'

In [5]:
model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 0.25, num_predict=1000 * 1, extract_reasoning=True)
error_model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 1, num_predict=1000 * 2)

In [6]:
system_template = DEEP_THINKING_INSTRUCTION + ("As trading technical analyst expert, your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "
                   "The timeframe of {stock} data is {timeframe}. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Thus, You MUST RESPECT the type of JSON schema entries. "
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                )

### pricesEvaluation

In [7]:
prices_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.PRICESEvaluation)

In [8]:
prices_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe"],
    partial_variables={"format_instructions": prices_evaluation_parser.get_format_instructions()}
)


In [9]:
# prices_retry_parser = RetryWithErrorOutputParser.from_llm(
#     llm=error_model,
#     parser=prices_evaluation_parser,
#     max_retries=3,
# )

In [10]:
prices_retry_parser = RetryWithErrorOutputParser(
    parser=prices_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | error_model | (lambda response: response.content),  # | (lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content)),
    max_retries=10,
)

In [11]:
prices_retry_parser.retry_chain

PromptTemplate(input_variables=['completion', 'error', 'prompt'], input_types={}, partial_variables={}, template='Enable deep thinking subroutine.\n\nPrompt:\n{prompt}\nCompletion:\n{completion}\n\nAbove, the Completion did not satisfy the constraints given in the Prompt.\nDetails: {error}\nYOU MUST RESPECT THE SCHEMA PROVIDED IN THE PROMPT.\nPlease try again:')
| ChatOllama(model='cogito:8b', num_ctx=4092, num_gpu=256, num_predict=2000)
| RunnableLambda(lambda response: response.content)

In [12]:
tmp_chain = prices_prompt_template | model | (lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))
tmp_response = tmp_chain.invoke(invocation)
tmp_response

"APPLE stands for:\n\nA - Apple Computer (company name)\nP - Personal computers\nP - PowerBook laptops\nL - Lisa computer line\nE - Established in 1976 by Steve Jobs, Steve Wozniak, and Ronald Wayne\n\nApple Inc. is a major technology company that designs, manufactures, and markets consumer electronics, software, and online services. It's known for products like the iPhone, iPad, Mac computers, Apple Watch, AirPods, and various software applications.\n\nWould you like to know more about any specific aspect of Apple?"

In [13]:
# prices_evaluation_parser.invoke(tmp_response)

In [14]:
tmp_result = prices_retry_parser.parse_with_prompt(completion=tmp_response, prompt_value=prices_prompt_template.format_prompt(**invocation))

Retry: 1
Completion <think>
Okay, let's break down what the user is asking for. They want a technical analysis report for Apple Inc., specifically formatted as a JSON object that follows their strict schema.

First, I need to recall the structure of the required JSON schema they provided. It has several key elements:

1. Prices trend
2. Trading action (BUY, OUTPERFORM, HOLD, UNDERPERFORM, SELL)
3. Primary prices trend category (STRONG_BULLISH, BULLISH, CONSOLIDATION, BEARISH, STRONG_BEARISH)
4. Number of touches for primary and secondary trends
5. Raw data value from the PRICES tool
6. Evaluation of the prices trend
7. Chart pattern description
8. Potential chart pattern to focus on
9. Candlestick pattern description
10. Potential candlestick pattern

Since it's a simulated analysis, I'll need to make up plausible values for each of these fields based on 5-minute timeframe data. The primary and secondary trends should be different, with their respective numbers of touches.

For the raw

In [15]:
tmp_result

PricesEvaluation(trend='STRONG_BULLISH', trading_action=<TradingActions.BUY: 'BUY'>, primary_trend=<TrendCategories.BULLISH: 'BULLISH'>, primary_trend_number_of_touches=3, secondary_trend=<TrendCategories.CONSOLIDATION: 'CONSOLIDATION'>, secondary_trend_number_of_touches=1, minor_trend=<TrendCategories.STRONG_BEARISH: 'STRONG_BEARISH'>, minor_trend_number_of_touches=2, raw_tool_data=PRICESRawValue(prices_value=172.45), trend_evaluation='Strong upward momentum with clear resistance levels', chart_pattern='Head and Shoulders reversal pattern forming at key resistance level', potential_chart_pattern='Double Top reversal setup expected if price breaks above $175.50', candlestick_pattern='Bullish Engulfing pattern on 5-minute timeframe suggesting strong buying pressure', potential_candlestick_pattern='Bearish Harami pattern forming, indicating possible trend weakness')

In [ ]:

support_chain = short_prompt_template | model | support_evaluation_parser
support_json = support_chain.invoke(invocation)
support_json

### RESISTANCEEvaluation

### PRICESEvaluation

### Indicators

### VOLUMESEvaluation